<a href="https://colab.research.google.com/github/danrosher/colab/blob/main/FineTuneSemanticSearchSymmetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install numpy datasets sentence_transformers transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:0

In [2]:

import numpy as np
from datasets import load_dataset

from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

from random import sample, seed, shuffle
from sentence_transformers import InputExample, losses, evaluation
from torch.utils.data import DataLoader
import pandas as pd

In [51]:
#model_name='msmarco-MiniLM-L12-cos-v5'
#model_name='all-MiniLM-L6-v2'
model_name='all-mpnet-base-v2'
#model_name='multi-qa-distilbert-cos-v1'
sbert_model = SentenceTransformer(model_name)
#sbert_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
#sbert_model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
sbert_model.max_seq_length = 256



In [52]:
from google.colab import drive

drive.mount('/g')

Drive already mounted at /g; to attempt to forcibly remount, call drive.mount("/g", force_remount=True).


In [53]:
df = pd.read_csv('/g/MyDrive/semantic_search/data.csv')


In [54]:
df.head()

,k,title,description,label
0,warehouse operative,Warehouse Operative (Nights),New4all Recruitment are hiring on behalf of a ...,1
1,warehouse operative,ELECTRICIAN,We require a qualified Electrician to work on ...,0
2,warehouse operative,Warehouse Operative,Red Recruitment Group urgently require a numbe...,1
3,warehouse operative,Class 1 HGV Training & Placement Programme,Do you enjoy driving?\nâ¢ Would you like to w...,0
4,warehouse operative,Factory Operative,"Are you hard working, reliable and looking for...",1


In [55]:
def convert_data(df):
  return list(df.apply(lambda x: InputExample(texts = (x['k'], x['title'][:800]),label = float(x['label'])), axis = 1)) 

In [56]:
good_training_data = df[df['label']==1].sample(n=2000)
bad_training_data = df[df['label']==0].sample(n=2000)

In [57]:
#training_data = df.sample(frac=1) #return all in random order
sampled_training_data = convert_data(pd.concat([good_training_data,bad_training_data]).sample(frac=1))
len(sampled_training_data)

4000

In [58]:
sampled_training_data[0].__dict__

{'guid': '', 'texts': ('executive assistant', 'Wood Machinist'), 'label': 0.0}

In [59]:
# Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(sampled_training_data, shuffle=True, batch_size=64)

train_loss = losses.CosineSimilarityLoss(sbert_model)

In [60]:
training_index = int(.8 * len(sampled_training_data))  # Get an 80/20 train/test split
train_examples = sampled_training_data[:training_index]


In [61]:
# Evaluation data
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(sampled_training_data[training_index:], main_similarity=evaluation.SimilarityFunction.COSINE)
sbert_model.evaluate(evaluator)

0.6491441704721985

In [62]:
# Tune the model
sbert_model.fit(
    train_objectives=[(train_dataloader, train_loss)], 
    output_path='/g/MyDrive/semantic_search/model',
    save_best_model=True,
    epochs=10,
#     warmup_steps=len(sampled_training_data) // 5, 
    evaluator=evaluator, 
    evaluation_steps=100
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

In [65]:
evaluator(sbert_model,output_path='/g/MyDrive/semantic_search_symmetric')

0.7180845312927072

In [64]:
sbert_model.save('/g/MyDrive/semantic_search_symmetric')


Now we create a **student** model  [see here](https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/distillation/dimensionality_reduction.py)

In [66]:
training_data = df[df['label']==1].sample(n=20000)


In [68]:
training_data

,k,title,description,label
35076,automation,Automation Tester,\n\nAutomation Tester âÂ Remote -Â Up to Â£...,1
38714,driving hgv2,Class 2 Day Drivers,Class 2 Day Drivers required for early starts ...,1
42080,director of estates,Director of Estates,Director of Estates\nCompetitive salary plus e...,1
17472,tooling engineer,Production Engineer,"Production Engineer â Â£30,000 - Â£40,000 - ...",1
10860,"(welding or ""qa/qc"" or ""quality control"" or ""q...",Quality Inspector,Matchtech are excited to be working alongside ...,1
...,...,...,...,...
2533,it project manager,Digital Project Manager - Amazing Rate - 80% R...,Gain valuable experience while impacting the l...,1
23357,data lead,Data Platforms Lead,"As the data platforms lead, you'll be responsi...",1
43025,rc frame,Setting Out Engineer,Lonsite is looking for Setting Out Engineer in...,1
2748,driver class 2,Hgv 2 Driver,Full Job Description\n\n***NEW PAY RATE!!**\n\...,1


In [106]:
new_dimension  = 32

In [107]:
student_model = SentenceTransformer('/g/MyDrive/semantic_search_symmetric')


In [108]:
train_embeddings = student_model.encode(training_data['title'].values.tolist(), convert_to_numpy=True)


In [109]:
#Compute PCA on the train embeddings matrix
from sklearn.decomposition import PCA
import torch


pca = PCA(n_components=new_dimension)
pca.fit(train_embeddings)
pca_comp = np.asarray(pca.components_)

In [110]:
# We add a dense layer to the model, so that it will produce directly embeddings with the new size
from sentence_transformers import models

dense = models.Dense(in_features=student_model.get_sentence_embedding_dimension(), out_features=new_dimension, bias=False, activation_function=torch.nn.Identity())
dense.linear.weight = torch.nn.Parameter(torch.tensor(pca_comp))
student_model.add_module('dense', dense)

In [111]:
# Evaluate the model with the reduce embedding size
evaluator(student_model)

0.7182903234145595

In [112]:
#student_model.save('/g/MyDrive/semantic_search_symmetric_student')
